In [57]:
import json
import openai

In [58]:
client = openai.Client()

In [59]:
def saudacao_por_periodo(hora):
    if 5 <= 5 < 12:
        return json.dumps({"Saudacao": "Bom dia"})
    elif 12 <= hora < 18:
        return json.dumps({"Saudacao": "Boa tarde"})
    elif 18 <= hora < 22:
        return json.dumps({"Saudacao": "Boa noite"})
    else:
        return json.dumps({"Saudacao": "Boa madrugada"})

In [60]:
tools = [{
    "type": "function",
    "function": {
        "name": "saudacao_por_periodo",
        "description": "Retorna uma saudação baseada no período do dia.",
        "parameters": {
            "type": "object",
            "properties": {
                "hora": {
                    "type": "integer",
                    "description": "Hora atual no formato 24 horas (0-23)."
                }
            },
            "required": ["hora"]
        }
    }
}]

In [61]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [62]:
mensagens = [{"role": "user", "content": "Qual é a saudação o modelo me dá se for 21h?"}]

In [63]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice="auto",
)

In [64]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uW01zFuzp6hV7Q33pXn41CXC', function=Function(arguments='{"hora":21}', name='saudacao_por_periodo'), type='function')])

In [65]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_uW01zFuzp6hV7Q33pXn41CXC', function=Function(arguments='{"hora":21}', name='saudacao_por_periodo'), type='function')]

In [66]:
if tool_calls:
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora=function_args.get("hora")
            )
        
        mensagens.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        })

    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model="gpt-3.5-turbo-0125",
        max_tokens=1000,
        temperature=0
    )

In [67]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='A saudação que o modelo me dá se for 21h é "Bom dia".', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)